In [1]:
!pip install selenium

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

## Selenium

In [125]:
# Open Shopee by Selenium
driver = webdriver.Chrome()
driver.get('https://shopee.co.th/')

In [126]:
thai_button = driver.find_element("xpath", '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

In [127]:
close_button = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_button.click()

In [128]:
search = driver.find_element("xpath",'/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('ชุดนักเรียนตรา')

In [129]:
search.send_keys(Keys.ENTER)

In [130]:
driver.execute_script("document.body.style.zoom='10%'")

## Scraping

In [181]:
Product = []
Price = []
Province = []
Sales = []

# Scrap 5 หน้า
for i in range(5):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el = soup.find_all('div', class_='VTjd7p whIxGK')
    for e in el:
        #Product name
        product = e.find('div', class_='ie3A+n bM+7UW Cve6sh').text
        product = re.sub('\u200b', '' ,product)
        
        # clean emoji
        emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
        product = emoji_pattern.sub(r'', product) # no emoji
        
        Product.append(product)
        
        #ราคา
        price = e.find('div', class_='hpDKMN').text
        # cleansing data , delect '-' only use lower price
        # clean ฿ and ,
        price = price.replace('฿' , '').replace(' ','')
        try:
            #ถ้าเป็นราคาธรรมดาให้เป็นทศนิยม
            start_price = float(price)
        except:
            #ถ้าราคาเป็นช่วงให้เอาตัวเลขก่อน - 
            price = price.rpartition('-')[0]
            start_price = float(price)
            
        Price.append(start_price)
        
        #จังหวัด
        province = e.find('div', class_='zGGwiV').text
        # cleansing data ตัดคำว่า''จังหวัด''
        province = province.replace("จังหวัด", "")
        Province.append(province)
        
        #ขายไปแล้ว ... ชิ้น
        sales = e.find('div', attrs={'class':'r6HknA'}).text.replace("พัน" , "000").replace("หมื่น", "0000")
        try:
            # cleansing data ที่ไม่ใช่ตัวเลขเราจะไม่เอา
            sales = float(re.sub('\D' ,'', sales))
        except:
            # ถ้ายังขายไม่ได้ให้ใส่ 0
            sales = 0
        Sales.append(sales) 
        
        # เพิ่มเข้าไปใน list ของ records
        #records.append([product,start_price,province,sales])
    
    time.sleep(3)
    # ปุ่ม next
    next_button = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[8]')
    # กด ปุ่ม next
    next_button.send_keys(Keys.ENTER)
    time.sleep(3)

In [183]:
# จำนวนสินค้าที่ Scrap ได้
len(records)

300

In [189]:
# Create DataFrame
df = {'Product': Product,
        'Province':Province,
        'Price': Price ,
        'Sales': Sales}

df = pd.DataFrame(df)
df

,Product,Province,Price,Sales
0,#เอี๊ยมคอกระเช้า #6 จีบ พร้อมเข็มขัด สีกรมอ่อน...,กรุงเทพมหานคร,250.0,8.0
1,กางเกงนักเรียนสีกรมท่า กางเกงนักเรียน ชุดนักเร...,กรุงเทพมหานคร,190.0,8.0
2,เสื้อนักเรียนหญิง ตราV.School เสื้อเนตรนารี ชุ...,สระบุรี,220.0,18.0
3,ชุดนักเรียนอนุบาลหญิงตราเรียนเก่ง (ได้ทั้งชุด ...,นครราชสีมา,185.0,30.0
4,กางเกงนักเรียน ชุดนักเรียน ตราสมาร์ท SMART สีด...,ขอนแก่น,200.0,7.0
...,...,...,...,...
295,วอกเกิ้ล ลูกเสือ เนตรนารี ลูกเสือสำรอง ลูกเสือ...,กรุงเทพมหานคร,10.0,9.0
296,กางเกงนักเรียนสีดำ กางเกงขาสั้นสีดำ ตราสมอ ชุ...,ลพบุรี,255.0,36.0
297,กางเกงนักเรียนตราสมอ กางเกงอนุบาล สีแดง เอวจั้...,ยโสธร,189.0,5.0
298,กระโปรงอนุบาล ตราสมอ ชุดนักเรียน กระโปรงนักเรี...,สงขลา,175.0,118.0


In [190]:
# check null and data types
print(df.isna().sum())
print('-------------------------')
print(df.dtypes)

Product     0
Province    0
Price       0
Sales       0
dtype: int64
-------------------------
Product      object
Province     object
Price       float64
Sales       float64
dtype: object


In [191]:
# Data Cleansing ลบสินค้าที่ไม่เกี่ยวข้องออกไป
df = df.drop(index = (df[df['Product'].str.contains("ญี่ปุ่น|คอสเพลย์|cosplay|ดอกไม้|sexy|เซ็กซี่")].index))
df = df.drop(index = (df[df['Province'].str.contains("ต่างประเทศ")].index))

In [192]:
# เอา # ที่อยู่ในชื่อ Product ออก
df['Product'] = df['Product'].replace(to_replace='#', regex=True, value='')

In [193]:
df = df.to_dict('records')
df

[{'Product': 'เอี๊ยมคอกระเช้า 6 จีบ พร้อมเข็มขัด สีกรมอ่อน ผ้าวิสด๊อป ตราช้างแมมมอธ ชุดนักเรียน เอี๊ยม กระโปรงนักเรียน',
  'Province': 'กรุงเทพมหานคร',
  'Price': 250.0,
  'Sales': 8.0},
 {'Product': 'กางเกงนักเรียนสีกรมท่า กางเกงนักเรียน ชุดนักเรียน ชั้นประถมศึกษา ชั้นมัธยมศึกษา ตรา เชาวไร',
  'Province': 'กรุงเทพมหานคร',
  'Price': 190.0,
  'Sales': 8.0},
 {'Product': 'เสื้อนักเรียนหญิง ตราV.School เสื้อเนตรนารี ชุดนักเรียนหญิง ชั้นประถม ชั้นมัธยมต้น ของเเท้100% ของบริษัทฯ ราคาส่ง บิลรร.',
  'Province': 'สระบุรี',
  'Price': 220.0,
  'Sales': 18.0},
 {'Product': 'ชุดนักเรียนอนุบาลหญิงตราเรียนเก่ง (ได้ทั้งชุด +มีบิลเบิกโรงเรียน)',
  'Province': 'นครราชสีมา',
  'Price': 185.0,
  'Sales': 30.0},
 {'Product': 'กางเกงนักเรียน ชุดนักเรียน ตราสมาร์ท SMART สีดำ ยาว 16,17,18,19,20 นิ้ว กางเกงนักเรียนชาย',
  'Province': 'ขอนแก่น',
  'Price': 200.0,
  'Sales': 7.0},
 {'Product': 'เสื้อนักเรียนหญิง ตราสมอ เสื้อปกทหารเรือ ชุดนักเรียนหญิง ชั้นประถม ชั้นมัธยมต้น ของเเท้100% ของบริษัทฯ ราคาส่ง บิลเบ

In [87]:
# save DataFeame to CSV
#df = df.to_csv('shopee_studentuniform_3.csv', index = False,  encoding ='utf-8-sig')

In [ ]:
# สำหรับการ save .json
import json
with open('shopee_studentuniform_3.json', 'w', encoding ='utf8') as json_file:
    json.dump(df, json_file)